In [ ]:
# Install dependencies.
!pip install asyncio==3.4.3 asyncpg==0.27.0 cloud-sql-python-connector["asyncpg"]==1.2.3
!pip install numpy==1.22.4 pandas==1.5.3
!pip install pgvector==0.1.8
!pip install langchain==0.0.196 transformers==4.30.1
!pip install google-cloud-aiplatform==1.26.0


In [1]:
# @markdown Replace the required placeholder text below. You can modify any other default values, if you like.

# Please fill in these values.
project_id = "imrenagi-gemini-experiment"  # @param {type:"string"}
database_password = "password01"  # @param {type:"string"}
region = "us-central1"  # @param {type:"string"}
instance_name = "pycon-demo"  # @param {type:"string"}
database_name = "courses-ai"  # @param {type:"string"}
database_user = "courses-ai-admin"  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"
assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"

In [ ]:
#@markdown ###Authenticate your Google Cloud Account and enable APIs.
# Authenticate gcloud.
# from google.colab import auth
# auth.authenticate_user()

# Configure gcloud.
!gcloud config set project {project_id}

# Grant Cloud SQL Client role to authenticated user
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

print(f"Granting Cloud SQL Client role to {current_user[0]}")

!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

# Enable Cloud SQL Admin API
!gcloud services enable sqladmin.googleapis.com
!gcloud services enable aiplatform.googleapis.com

In [ ]:
#@markdown Create and setup a Cloud SQL PostgreSQL instance, if not done already.
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found an existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  !gcloud sql instances create {instance_name} --database-version=POSTGRES_15 \
    --region={region} --cpu=1 --memory=4GB --root-password={database_password}

# Create the database, if it does not exist.
out = !gcloud sql databases list --instance={instance_name} --filter="NAME:{database_name}" --format="value(NAME)"
if ''.join(out) == database_name:
  print("Database %s already exists, skipping creation." % database_name)
else:
  !gcloud sql databases create {database_name} --instance={instance_name}

# Create the database user for accessing the database.
!gcloud sql users create {database_user} \
  --instance={instance_name} \
  --password={database_password}

In [ ]:
# get the ip address of the instance

!gcloud sql instances describe {instance_name} --project {project_id} --format 'value(ipAddresses.ipAddress)'

In [ ]:
# !gcloud sql instances delete {instance_name} --project={project_id} --quiet --region={region}